* Ray RLlib 노트북

필요 패키지 삽입

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

from horcrux_terrain_v1.envs import SandWorld
from horcrux_terrain_v1.envs import PlaneWorld

import ray
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig

from ray.tune.registry import register_env


c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Ray 실행 (Warning 관련 무시 키워드)

In [2]:
ray.init(runtime_env={"env_vars": {"PYTHONWARNINGS": "ignore::DeprecationWarning"}})

2024-10-03 11:50:41,273	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.5
Ray version:,2.36.1
Dashboard:,http://127.0.0.1:8265


Gym -> Rllib Env 등록

In [3]:
env_config = {
    "forward_reward_weight": 4,
    "side_cost_weight": 2,
    "unhealthy_max_steps": 75,
    "healthy_roll_range": (-55,55),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.05,
    "termination_reward": 0,
}

# Sand
register_env("sand-v1", lambda config: SandWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 terminating_roll_range=env_config["terminating_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 termination_reward=env_config["termination_reward"]))

# Plane
register_env("plane-v1", lambda config: PlaneWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 terminating_roll_range=env_config["terminating_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 termination_reward=env_config["termination_reward"]))


학습 알고리즘 설정

In [4]:
config = PPOConfig()
# Activate new API stack. -> 구려서 안씀.
config.api_stack(
    enable_rl_module_and_learner=False,
    enable_env_runner_and_connector_v2=False,
)
# config.environment("sand-v1")
config.environment("plane-v1")
config.framework("torch")
total_workers = 12
config.resources(num_gpus=1,num_cpus_per_worker=1, num_gpus_per_worker= 1/(total_workers+1))
config.rollouts(num_rollout_workers=total_workers)
config.training(
    gamma=0.9, 
    lr=0.0001, 
    # kl_coeff=0.3, 

    # See model catalog for more options.
    # https://docs.ray.io/en/latest/rllib/rllib-models.html
    model={ "fcnet_hiddens": [512, 512, 512, 512, 512],
            },
)
config.evaluation(evaluation_interval=100)

# # See model catalog for more options.
# # https://docs.ray.io/en/latest/rllib/rllib-models.html
# # config.model["fcnet_hiddens"] = [512, 512, 512, 512, 512]
# config.model["uses_new_env_runners"] = True
# config.model["fcnet_hiddens"] = [1024, 1024, 1024, 1024, 1024]
# config.model["use_lstm"] = True
# # config.model["lstm_cell_size"] = 2048
# config.model["lstm_cell_size"] = 4096
# config.model["max_seq_len"] = 200
# config.model["lstm_use_prev_action"] = True

algo = config.build()



2024-10-03 11:51:00,108	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.resources(num_cpus_per_worker)` has been deprecated. Use `AlgorithmConfig.env_runners(num_cpus_per_env_runner)` instead. This will raise an error in the future!
2024-10-03 11:51:00,108	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.resources(num_gpus_per_worker)` has been deprecated. Use `AlgorithmConfig.env_runners(num_gpus_per_env_runner)` instead. This will raise an error in the future!
2024-10-03 11:51:00,108	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-10-03 11:51:00,108	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\rll

혹시 이전 학습 결과를 로드할 경우

In [ ]:
algo = Algorithm.from_checkpoint("./PPO/GD_v2/1st/PPO_GD_V2_512x5_final")

학습 파라미터 재조정

In [6]:
# algo.get_config().training().num_sgd_iter

#Env runner 파라미터 보기.
# algo.env_runner.config["exploration_config"]
# algo.get_config().model

# algo.compute_single_action()
# algo.get_policy()

학습 시작

In [5]:
from pprint import pprint

n_iter = 1000
save_iter = 0
save_name = "PPO_GD_V3_512x5"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}th iteration done")
    # result.pop("config")
    # pprint(result)

    if i%60 == 0:
        checkpoint_dir = algo.save(save_name+"_"+str(save_iter))
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1

algo.save(save_name+str("_final"))

2024-10-03 11:52:21,634	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


000th iteration done
Checkpoint saved in directory TrainingResult(checkpoint=Checkpoint(filesystem=local, path=PPO_GD_V3_512x5_0), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.00010000000000000003, 'total_loss': 1.2339975607931934, 'policy_loss': -0.18959970952722655, 'vf_loss': 1.4140552848257044, 'vf_explained_var': 0.23740630976615412, 'kl': 0.047709940831922995, 'entropy': 19.856002411791074, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'env_runners': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_timesteps_total': 0, 'policy_reward_min': {}, 'policy_rew

2024-10-03 12:09:52,513	WARNING env_runner_v2.py:158 -- More than 6000 observations in 6000 env steps for episode 728404237681758076 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.


099th iteration done
100th iteration done
101th iteration done
102th iteration done
103th iteration done
104th iteration done
105th iteration done
106th iteration done
107th iteration done
108th iteration done
109th iteration done
110th iteration done
111th iteration done
112th iteration done
113th iteration done
114th iteration done
115th iteration done
116th iteration done
117th iteration done
118th iteration done
119th iteration done
120th iteration done
Checkpoint saved in directory TrainingResult(checkpoint=Checkpoint(filesystem=local, path=PPO_GD_V3_512x5_2), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 1.5187500000000005, 'cur_lr': 0.00010000000000000003, 'total_loss': 0.25191531026315306, 'policy_loss': -0.20997085455224238, 'vf_loss': 0.43917697154183283, 'vf_explained_var': 0.6513768991475464, 'kl': 0.014952555193076967, 'entropy': 19.47429323709139, 'entropy_coeff': 0.0}, '

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=PPO_GD_V3_512x5_final), metrics={'evaluation': {'env_runners': {'episode_reward_max': 5723.757065302427, 'episode_reward_min': 5576.7617990429, 'episode_reward_mean': 5637.427962705935, 'episode_len_mean': 6000.0, 'episode_media': {}, 'episodes_timesteps_total': 60000, 'policy_reward_min': {'default_policy': 5576.7617990429}, 'policy_reward_max': {'default_policy': 5723.757065302427}, 'policy_reward_mean': {'default_policy': 5637.427962705935}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [5577.758627678155, 5636.487975045921, 5576.7617990429, 5622.9884634896025, 5664.3256521857575, 5660.9263460111815, 5607.499493993334, 5590.432282006756, 5713.34192230332, 5723.757065302427], 'episode_lengths': [6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000, 6000], 'policy_default_policy_reward': [5577.758627678155, 5636.487975045921, 5576.7617990429, 5622.9884634896025, 5664.3256521857575, 5660.9263460111815, 5607.4994939933

환경에서 학습된 Policy 테스트하기 (RL Module 사용)

In [ ]:
from ray.rllib.core.rl_module import RLModule
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
import time

algo = Algorithm.from_checkpoint("./ModelV2_512x5_11")


In [ ]:
# env = gym.make("horcrux_terrain_v1/plane-v1", 
#                terminate_when_unhealthy = False, 
#                render_mode = "human", 
#             #    render_camera_name = 'ceiling', 
#                use_gait = True,
#                gait_params = (30,30,40,40,0),
#                **env_config,
#                ) 

# obs, info = env.reset()

# algo.get_policy().action_connectors

In [ ]:
# env = gym.make("horcrux_terrain_v1/sand-v1", 
#                terminate_when_unhealthy = False, 
#                render_mode = "human", 
#             #    render_camera_name = 'ceiling', 
#                use_gait = True,
#                gait_params = (30,30,40,40,0),
#                **env_config,
#                ) 

env = gym.make("horcrux_terrain_v1/plane-v1", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
            #    render_camera_name = 'ceiling', 
               use_gait = True,
               gait_params = (30,30,40,40,0),
               **env_config,
               ) 

for j in range(5):
   episode_return = 0
   terminated = truncated = False

   obs, info = env.reset()


   for i in range(6000):
      action = algo.compute_single_action(obs, explore=False)
      
      obs, reward, terminated, truncated, info = env.step(action)
      
      prev_a = action

      if terminated:
         print("terminated")

      episode_return += reward

   print(f"Reached episode return of {episode_return}.")

env.close()

환경에서 학습된 Policy 테스트하기 (PPO 알고리즘 사용)

In [ ]:
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld
from ray.rllib.algorithms.algorithm import Algorithm
import time

env = gym.make("horcrux_terrain_v1/plane-v1", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
            #    render_camera_name = 'ceiling', 
               use_gait = True,
               gait_params = (30,30,40,40,0),
               **env_config,
               ) 

for j in range(10):
   episode_return = 0
   terminated = truncated = False

   obs, info = env.reset()

   init_state = state = [np.zeros([4096], np.float32) for _ in range(200)]
   prev_action = np.zeros((14), np.float32)
   for i in range(1000):

      a, init_state = algo.compute_single_action(observation= obs, state=init_state, prev_action=prev_action,policy_id="default_policy")
      
      obs, reward, terminated, truncated, info = env.step(action)

      prev_action = a
      if terminated:
         print("terminated")

      episode_return += reward

   print(f"Reached episode return of {episode_return}.")

env.close()